In [263]:
from shared import const
from shared import preproc
import numpy as np
import pandas as pd
import re

# Gensim
import gensim
from gensim import models
import gensim.corpora as corpora
from gensim.utils import simple_preprocess, lemmatize
from gensim.models import CoherenceModel, LdaModel, LdaMulticore

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
#matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

#Choose Random forest for multiclass classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import ComplementNB
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score, f1_score


In [303]:
#prepare stopwords
from nltk.corpus import stopwords

stop_words = stopwords.words("english")

#for the purpose of emotion classification, we trim the words by extending stopwords:
stop_words.extend(["from", "re", "use", "day", "let", "get","say", "know", "think", "love"])

In [387]:
#read data
df_full = pd.read_csv(const.CLEAN_DEEZER)
print(df_full.shape)
df_1 = preproc.get_class_based_data(df_full, 1) #
df_2 = preproc.get_class_based_data(df_full, 2)
df_3 = preproc.get_class_based_data(df_full, 3)
df_4 = preproc.get_class_based_data(df_full, 4)
df_1.reset_index()
df_2.reset_index()
df_3.reset_index()
df_4.reset_index()
Y1 = df_1.iloc[:TRAIN_SAMPLE, -1]
Y2 = df_2.iloc[:TRAIN_SAMPLE, -1]
Y3 = df_3.iloc[:TRAIN_SAMPLE, -1]
Y4 = df_4.iloc[:TRAIN_SAMPLE, -1]
print(df_3.shape)

(15516, 8)
(5730, 8)


In [388]:
#preproccesing

#remove words fewer than 3 characters
def remove_words(list2d_words, threshold = 3):
    '''
    return a 2d list of words
    '''
    words_more_than_3chars = []
    for i in range(len(list2d_words)):
        proc_sentence = []
        for j in range(len(list2d_words[i])):
            if len(list2d_words[i][j]) >= threshold:
                proc_sentence.append(list2d_words[i][j])
        words_more_than_3chars.append(proc_sentence)
    return words_more_than_3chars
        

#remove stopwords
#simple_preprocess will remove punctuations and unnecessary characters altogether
def remove_stopwords(list2d_words):
    return [[word for word in simple_preprocess(" ".join(doc)) if word not in stop_words] for doc in list2d_words]

#lemmatize: word in third person changed to first person, etc, and only keep the nouns, adj, verb, adv
nlp = spacy.load('en', disable = ['parse', 'ner'])

def my_lemmatize(list2d_words, allowed_postags = ['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in list2d_words:
        doc = nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

TRAIN_SAMPLE = int(len(df_1))
train_dflist_1 = df_1.lyrics[:TRAIN_SAMPLE].tolist()
train_dflist_2 = df_2.lyrics[:TRAIN_SAMPLE].tolist()
train_dflist_3 = df_3.lyrics[:TRAIN_SAMPLE].tolist()
train_dflist_4 = df_4.lyrics[:TRAIN_SAMPLE].tolist()

In [389]:
#tokenize each document into a list of words, removing punctuations and unnecessary characters altogether.
def sentences_to_words(sentences):
    yield(gensim.utils.simple_preprocess(str(sentences), deacc=True))  # deacc=True removes punctuations

import itertools

def corpus_to_2d_words(corpus):
    word_corpus = []
    for i in range(len(corpus)):
        data_words = list(sentences_to_words(corpus[i]))
        word_corpus = [*word_corpus, *data_words]
    return word_corpus

def words_2d_to_dict(word_2d_list):
    dictionary = corpora.Dictionary(word_2d_list)
    return dictionary

def df_to_lemmatized_words(df):
    words_2d = corpus_to_2d_words(df)
    words_morethan_3chars = remove_words(words_2d)
    words_nostops = remove_stopwords(words_morethan_3chars)
    lemmatized_words = my_lemmatize(words_nostops)
    return lemmatized_words

def lemmatized_to_corpus(words_2d):
    dict_words = words_2d_to_dict(words_2d)
    corpus = [dict_words.doc2bow(simple_preprocess(" ".join(line))) for line in words_2d]
    
def build_LDA_model(dict_words, corpus):
    lda_model = gensim.models.ldamodel.LdaModel(corpus = corpus,
                                          id2word = dict_words,
                                          num_topics = NUM_TOPICS, #tried 3, 4, 5, 6, 10 , 20. seems 3 for coherence is the best, the second is 4
                                          random_state = 100, #the seed 
                                          update_every = 0, #batch mode
                                          chunksize = 1000, #larger the heavier workload for the memory
                                          passes = 10,#how to determine the chunksize and passes?relevant to size of dataset?
                                          alpha = 'auto',
                                          minimum_probability = 0.0,
                                          per_word_topics = False #non-indicative words would be omitted
                                          )
    return lda_model
    
#evaluate by perplexity and coherence
def print_coherence_perplexity(the_model, lemmatized_words, dict_words, corpus, f=None):
    coherence_model = CoherenceModel(model = the_model, texts = lemmatized_words, dictionary = dict_words, coherence = 'c_v')
    coherence_lda = coherence_model.get_coherence()
    #compute model coherence: higher the better:
    print('\nCoherence Score: ', coherence_lda, file = f)
    #compute model perplexity: lower the better:
    print('\nPerplexity: ', the_model.log_perplexity(corpus), file = f)
    
#get the LDA generated topic-document dataset and responding label for classification
def generate_data(model, corpus):
    topics_features = []
    for i in range(TRAIN_SAMPLE):
        row = []
        for j in range(NUM_TOPICS):
            row.append(model.get_document_topics(corpus[i])[j][1])
        topics_features.append(row)
    #generate data for supervised classification
    gen_df_mid = pd.DataFrame(topics_features)
    return gen_df_mid

def generate_train_test(X, Y):
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0)
    return X_train, X_test, y_train, y_test

#z-score normalization
def zscore_normalization(X_train, X_test):
    zscoreScaler = StandardScaler().fit(X_train)
    X_tr_std = zscoreScaler.transform(X_train)
    X_ts_std = zscoreScaler.transform(X_test)
    return X_tr_std, X_ts_std

def RF_classify_score(X_tr_std, X_ts_std, y_train, y_test):
    rf_clf = RandomForestClassifier()
    rf_clf.fit(X_tr_std, y_train)
    rf_pred = rf_clf.predict(X_ts_std)
    score = accuracy_score(y_test, rf_pred)
    print(score)
    return score

def complement_classify_score(X_train, X_test, y_train, y_test):
    cnb = ComplementNB(alpha = 0.1)
    cnb.fit(X_train, y_train)
    cnb_pred = cnb.predict(X_test)
    score = accuracy_score(y_test, cnb_pred)
    print(score)
    return score
    
    
    
    
def randomForest_LDA(X, Y):
    X1_train, X1_test, y1_train, y1_test = generate_train_test(X,Y)
    #z-score normalization
    X1_tr_std, X1_ts_std = zscore_normalization(X1_train, X1_test)
    RF_classify_score(X1_tr_std, X1_ts_std, y1_train, y1_test)
    
def complementNB_LDA(X, Y):
    X_train, X_test, y_train, y_test = generate_train_test(X,Y)
    #X_tr_std, X_ts_std = zscore_normalization(X_train, X_test)
    complement_classify_socre(X_train, X_test, y_train, y_test)


In [390]:
#use k-fold validation:
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
kFolder = KFold(n_splits = 10, shuffle = True) #5-fold cross-validation

def kfold_classification(X, Y, clf, normalization_func = None, f = None):
    sum_accuracy = []
    for k, (train, test) in enumerate(kFolder.split(X, Y)):
        X_train = np.array(X.iloc[train, :])
        X_test = np.array(X.iloc[test, :])
        y_train = np.array(Y.iloc[train])
        #print(y_train.shape)
        y_test = np.array(Y.iloc[test])
        if normalization_func != None:
            X_train, X_test = normalization_func(X_train, X_test)
        #rf_clf = RandomForestClassifier()
        clf.fit(X_train, y_train)
        rf_pred = clf.predict(X_test)
        accuracy = accuracy_score(y_test, rf_pred)
        sum_accuracy.append(accuracy)
        #print("the {} th fold split got accuracy {}; f1-score: {} ".format(k, accuracy, f1_score(y_test, rf_pred)), file = f)
    k_fold_avg = sum(sum_accuracy)/len(sum_accuracy)
    print("the average of 10 rounds is {}".format(k_fold_avg), file = f)
    return k_fold_avg

In [ ]:
lemmatized_words_1 = df_to_lemmatized_words(train_dflist_1)
lemmatized_words_2 = df_to_lemmatized_words(train_dflist_2)
lemmatized_words_3 = df_to_lemmatized_words(train_dflist_3)
lemmatized_words_4 = df_to_lemmatized_words(train_dflist_4)

#print(lemmatized_words)

In [ ]:
#turn the 2d word list to dict 
dict_words_1 = words_2d_to_dict(lemmatized_words_1)
dict_words_2 = words_2d_to_dict(lemmatized_words_2)
dict_words_3 = words_2d_to_dict(lemmatized_words_3)
dict_words_4 = words_2d_to_dict(lemmatized_words_4)
#print(dict_words)

In [ ]:
#there's 2 input for lDA: dictinary and the corpus
#create bag of words
corpus_1 =  [dict_words_1.doc2bow(simple_preprocess(" ".join(line))) for line in lemmatized_words_1]
corpus_2 =  [dict_words_2.doc2bow(simple_preprocess(" ".join(line))) for line in lemmatized_words_2]
corpus_3 =  [dict_words_3.doc2bow(simple_preprocess(" ".join(line))) for line in lemmatized_words_3]
corpus_4 =  [dict_words_4.doc2bow(simple_preprocess(" ".join(line))) for line in lemmatized_words_4]

Several parameters to adjust:

the number of topics;

alpha and beta: hyperparameter that affect sparsity of the topics. Default to 1.0/num_topics

chunksize: the number of documents to be used in each training chunk

update_every: how often the model parameters should be updated

passes: the total number of training passes

In [ ]:
# # Show the Word Weights in Corpus
# num_row = 0
# for doc in corpus:#every row in corpus: id of every word in the doc and the freq of that word in the doc
#     num_row += 1
#     print([[dict_words[id], freq] for id, freq in doc])
#     print("\n{}".format(num_row),"---------------------------------")

#Gensim creates a unique id for each word in the document. The produced corpus shown above is a mapping of (word_id, word_frequency).

In [ ]:
#Build LDA model
#NUM_TOPICS = 8
f = open('out.txt', 'a')
for i in range(3, 8):
    NUM_TOPIC = i
    lda_model_1 = build_LDA_model(dict_words_1, corpus_1)
    lda_model_2 = build_LDA_model(dict_words_2, corpus_2)

    lda_model_3 = build_LDA_model(dict_words_3, corpus_3)

    lda_model_4 = build_LDA_model(dict_words_4, corpus_4)
    print("---------------------start of topic # {}".format(i), "-------------------", file = f)
    print_coherence_perplexity(lda_model_1, lemmatized_words_1, dict_words_1, corpus_1)
    
    print_coherence_perplexity(lda_model_2, lemmatized_words_2, dict_words_2, corpus_2)

    print_coherence_perplexity(lda_model_3, lemmatized_words_3, dict_words_3, corpus_3)

    print_coherence_perplexity(lda_model_4, lemmatized_words_4, dict_words_4, corpus_4)

    gen_df_mid_1 = generate_data(lda_model_1, corpus_1)

    gen_df_mid_2 = generate_data(lda_model_2, corpus_2)
    gen_df_mid_3 = generate_data(lda_model_3, corpus_3)
    gen_df_mid_4 = generate_data(lda_model_4, corpus_4)

    #randomForest_LDA(gen_df_mid_1, df_1.iloc[:TRAIN_SAMPLE, -1])

    #kfold for class 1:
    print("random forest classificaton:")
    rf_clf = RandomForestClassifier()
    rf_score_sum = kfold_classification(gen_df_mid_1, Y1, rf_clf, zscore_normalization)
    
    #randomForest_LDA(gen_df_mid_2, df_2.iloc[:TRAIN_SAMPLE, -1])
    #kfold for class 2:
    #random forest classificaton:
    rf_clf = RandomForestClassifier()
    rf_score_sum = rf_score_sum + kfold_classification(gen_df_mid_2, Y2, rf_clf, zscore_normalization)
    
    #randomForest_LDA(gen_df_mid_3, df_3.iloc[:TRAIN_SAMPLE, -1])
    #kfold for class 3:
    #random forest classificaton:
    rf_clf = RandomForestClassifier()
    rf_score_sum = rf_score_sum + kfold_classification(gen_df_mid_3, Y3, rf_clf, zscore_normalization)
    
    #randomForest_LDA(gen_df_mid_4, df_4.iloc[:TRAIN_SAMPLE, -1])
    #kfold for class 4:
    #random forest classificaton:
    rf_clf = RandomForestClassifier()
    rf_score_sum = rf_score_sum + kfold_classification(gen_df_mid_4, Y4, rf_clf, zscore_normalization)
    rf_score_avg = rf_score_sum / 4
    print("topics: {}, RF score average of 4 models: {}".format(i, rf_score_avg), file = f)

    print("complement naive bayes classifier:")
    cnb_clf = ComplementNB(alpha= 0.1)
    cnb_score_sum = kfold_classification(gen_df_mid_1, Y1, cnb_clf)
    
    #complement naive bayes classifier:
    cnb_clf = ComplementNB(alpha= 0.1)
    cnb_score_sum = cnb_score_sum + kfold_classification(gen_df_mid_2, Y2, cnb_clf)

    #complement naive bayes classifier:
    cnb_clf = ComplementNB(alpha= 0.1)
    cnb_score_sum = cnb_score_sum + kfold_classification(gen_df_mid_3, Y3, cnb_clf)
    
    #complement naive bayes classifier:
    cnb_clf = ComplementNB(alpha= 0.1)
    cnb_score_sum = cnb_score_sum + kfold_classification(gen_df_mid_4, Y4, cnb_clf)
    cnb_score_avg = cnb_score_sum / 4
    print("Complement NB score average of 4 models: {}".format(cnb_score_avg), file = f)
    print("---------------------end of topic # {}".format(i), "-------------------", file = f)
    f.flush()
f.close()

In [376]:
#visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model_1, corpus_1, dict_words_1)
vis

/usr/local/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.033493  0.021014       1        1  30.694830
0      0.073125  0.052225       2        1  25.095102
2      0.033172 -0.085341       3        1  22.745213
1     -0.139791  0.012102       4        1  21.464849, topic_info=     Category         Freq    Term        Total  loglift  logprob
term                                                             
213   Default   715.000000   bitch   715.000000  30.0000  30.0000
579   Default   597.000000   nigga   597.000000  29.0000  29.0000
300   Default   675.000000    shit   675.000000  28.0000  28.0000
279   Default   581.000000    move   581.000000  27.0000  27.0000
1020  Default   324.000000    boom   324.000000  26.0000  26.0000
619   Default  1484.000000    baby  1484.000000  25.0000  25.0000
723   Default   774.000000    fuck   774.000000  24.0000  24.0000
29    Default  1357.000000     way  1357.000000  23.0000  23.0000
84    Default   393.000000   dance   393.000000  22.0000  22.0000
258   Default   478.000000    high   478.000000  21.0000  21.0000
27    Default  1435.000000   wanna  1435.000000  20.0000  20.0000
439   Default   247.000000     que   247.000000  19.0000  19.0000
28    Default  1827.000000    want  1827.000000  18.0000  18.0000
285   Default   285.000000  niggas   285.000000  17.0000  17.0000
9     Default  1633.000000    feel  1633.000000  16.0000  16.0000
5141  Default   166.000000     doo   166.000000  15.0000  15.0000
134   Default   497.000000    home   497.000000  14.0000  14.0000
113   Default  1579.000000    take  1579.000000  13.0000  13.0000
90    Default   825.000000   heart   825.000000  12.0000  12.0000
10    Default  1130.000000    girl  1130.000000  11.0000  11.0000
166   Default   443.000000  change   443.000000  10.0000  10.0000
1180  Default   383.000000     fly   383.000000   9.0000   9.0000
1903  Default   232.000000    drip   232.000000   8.0000   8.0000
351   Default   226.000000     woo   226.000000   7.0000   7.0000
528   Default   200.000000     ass   200.000000   6.0000   6.0000
162   Default   538.000000   world   538.000000   5.0000   5.0000
19    Default   983.000000   night   983.000000   4.0000   4.0000
571   Default   180.000000     lil   180.000000   3.0000   3.0000
655   Default   513.000000   money   513.000000   2.0000   2.0000
259   Default   406.000000     hit   406.000000   1.0000   1.0000
...       ...          ...     ...          ...      ...      ...
723    Topic4   439.895508    fuck   774.469604   0.9731  -4.5665
300    Topic4   389.023468    shit   675.073608   0.9876  -4.6894
624    Topic4    64.521553   booty    79.812927   1.3261  -6.4860
259    Topic4   233.462708     hit   406.717010   0.9837  -5.2000
1697   Topic4    98.708305     bag   141.309662   1.1800  -6.0608
670    Topic4   143.687241   shake   234.638611   1.0483  -5.6854
655    Topic4   253.881073   money   513.231079   0.8349  -5.1161
853    Topic4   152.522644    drop   276.007202   0.9456  -5.6257
99     Topic4   141.819473    pull   251.947083   0.9641  -5.6984
43     Topic4   783.966125     get  3268.943359   0.1109  -3.9886
1349   Topic4   126.170006     ayy   240.341644   0.8943  -5.8154
251    Topic4   172.471100     gon   394.603088   0.7111  -5.5028
293    Topic4   205.892776     put   524.449463   0.6038  -5.3256
65     Topic4   205.366959    talk   538.319519   0.5751  -5.3282
32     Topic4   188.738007     boy   478.670593   0.6081  -5.4126
93     Topic4   234.068054     man   716.860352   0.4195  -5.1974
30     Topic4   290.166351    back  1118.687378   0.1893  -4.9825
106    Topic4   303.860626     see  1535.126465  -0.0810  -4.9364
130    Topic4   318.380310      go  2175.503174  -0.3830  -4.8898
7      Topic4   268.947815    come  1741.254639  -0.3291  -5.0585
252    Topic4   190.551788    good   805.369263   0.0974  -5.4031
1101   Topic4   180.334915     run   694.836731   0.1899  -5.4582
5  

In [377]:
vis = pyLDAvis.gensim.prepare(lda_model_2, corpus_2, dict_words_2)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.091740  0.019227       1        1  35.910278
2      0.065053 -0.051247       2        1  25.724512
0     -0.019137 -0.055887       3        1  20.218390
1      0.045824  0.087907       4        1  18.146820, topic_info=     Category         Freq      Term        Total  loglift  logprob
term                                                               
67    Default  2054.000000        go  2054.000000  30.0000  30.0000
299   Default   549.000000     dance   549.000000  29.0000  29.0000
89    Default   652.000000     nigga   652.000000  28.0000  28.0000
64    Default  3223.000000       get  3223.000000  27.0000  27.0000
286   Default   639.000000     light   639.000000  26.0000  26.0000
14    Default  1027.000000     night  1027.000000  25.0000  25.0000
99    Default   355.000000     shake   355.000000  24.0000  24.0000
62    Default   825.000000      fuck   825.000000  23.0000  23.0000
829   Default   344.000000     taste   344.000000  22.0000  22.0000
28    Default  1631.000000      baby  1631.000000  21.0000  21.0000
303   Default  1015.000000     heart  1015.000000  20.0000  20.0000
138   Default   640.000000   nothing   640.000000  19.0000  19.0000
642   Default   306.000000       fly   306.000000  18.0000  18.0000
83    Default   746.000000       man   746.000000  17.0000  17.0000
4337  Default   140.000000      boom   140.000000  16.0000  16.0000
96    Default   683.000000       run   683.000000  15.0000  15.0000
249   Default   268.000000  somebody   268.000000  14.0000  14.0000
33    Default   661.000000     bitch   661.000000  13.0000  13.0000
315   Default  1366.000000     wanna  1366.000000  12.0000  12.0000
86    Default   454.000000     money   454.000000  11.0000  11.0000
4593  Default   113.000000    bubble   113.000000  10.0000  10.0000
2811  Default   155.000000      wild   155.000000   9.0000   9.0000
849   Default   208.000000      jump   208.000000   8.0000   8.0000
527   Default  1128.000000      keep  1128.000000   7.0000   7.0000
100   Default   481.000000      shit   481.000000   6.0000   6.0000
4032  Default   111.000000      halo   111.000000   5.0000   5.0000
316   Default   252.000000       woo   252.000000   4.0000   4.0000
13    Default  1172.000000      need  1172.000000   3.0000   3.0000
90    Default   236.000000    niggas   236.000000   2.0000   2.0000
934   Default   143.000000   problem   143.000000   1.0000   1.0000
...       ...          ...       ...          ...      ...      ...
1032   Topic4    74.659805      less   113.039040   1.2919  -6.1660
317    Topic4    94.359253    afraid   155.796997   1.2052  -5.9318
64     Topic4   870.389282       get  3223.355469   0.3974  -3.7100
522    Topic4   146.977219     black   299.564880   0.9946  -5.4886
376    Topic4    90.392731     smoke   155.331360   1.1653  -5.9748
96     Topic4   256.222931       run   683.911987   0.7249  -4.9329
28     Topic4   469.969757      baby  1631.450928   0.4621  -4.3263
13     Topic4   348.408722      need  1172.780396   0.4929  -4.6256
583    Topic4   118.701164     drive   243.235931   0.9893  -5.7023
18     Topic4   363.716644       see  1634.945679   0.2037  -4.5826
1309   Topic4    65.824684      lady   109.655525   1.1963  -6.2919
65     Topic4   265.425385      girl  1045.341064   0.3359  -4.8976
9      Topic4   352.883545      feel  1738.219604   0.1122  -4.6128
146    Topic4   274.059570      tell  1284.736938   0.1617  -4.8656
237    Topic4   217.091858     right   876.309998   0.3113  -5.0986
151    Topic4   147.868011     world   468.371735   0.5537  -5.4826
1      Topic4   117.358482       boy   323.269562   0.6934  -5.7137
292    Topic4   275.931671      want  1706.133545  -0.1152  -4.8588
128    Topic4   227.350174     cause  1365.481812  -0.0861  -5.0524
82     Topic4   257.948730      make  1819.245483  -0.2467  -4.9262
29     Topic4   204.182022   

In [378]:
vis = pyLDAvis.gensim.prepare(lda_model_3, corpus_3, dict_words_3)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.048871  0.016528       1        1  38.828316
0      0.087725  0.053757       2        1  30.592363
1      0.015795 -0.096527       3        1  20.307251
2     -0.152392  0.026242       4        1  10.272062, topic_info=     Category         Freq     Term        Total  loglift  logprob
term                                                              
683   Default   338.000000    black   338.000000  30.0000  30.0000
645   Default   912.000000     girl   912.000000  29.0000  29.0000
81    Default  1710.000000     feel  1710.000000  28.0000  28.0000
520   Default   294.000000   better   294.000000  27.0000  27.0000
16    Default  1797.000000       go  1797.000000  26.0000  26.0000
45    Default  1201.000000     baby  1201.000000  25.0000  25.0000
335   Default   402.000000    nigga   402.000000  24.0000  24.0000
268   Default   398.000000    dance   398.000000  23.0000  23.0000
436   Default   185.000000   afraid   185.000000  22.0000  22.0000
283   Default   587.000000     fuck   587.000000  21.0000  21.0000
456   Default   420.000000     stop   420.000000  20.0000  20.0000
384   Default  1573.000000     want  1573.000000  19.0000  19.0000
4677  Default   102.000000     pump   102.000000  18.0000  18.0000
315   Default   166.000000      lil   166.000000  17.0000  17.0000
141   Default   301.000000  someone   301.000000  16.0000  16.0000
365   Default   389.000000     shit   389.000000  15.0000  15.0000
161   Default  1789.000000     come  1789.000000  14.0000  14.0000
3459  Default    92.000000   bounce    92.000000  13.0000  13.0000
14    Default  2729.000000      get  2729.000000  12.0000  12.0000
5584  Default   165.000000      doo   165.000000  11.0000  11.0000
3436  Default    79.000000      que    79.000000  10.0000  10.0000
239   Default   441.000000    bitch   441.000000   9.0000   9.0000
93    Default  1746.000000     make  1746.000000   8.0000   8.0000
3336  Default    75.000000     halo    75.000000   7.0000   7.0000
383   Default  1180.000000    wanna  1180.000000   6.0000   6.0000
474   Default   259.000000  forever   259.000000   5.0000   5.0000
224   Default   257.000000    wrong   257.000000   4.0000   4.0000
1945  Default    63.000000    beast    63.000000   3.0000   3.0000
1616  Default    62.000000    ounce    62.000000   2.0000   2.0000
1407  Default   136.000000    gimme   136.000000   1.0000   1.0000
...       ...          ...      ...          ...      ...      ...
3899   Topic4    53.636459    ahead    82.186127   1.8490  -5.8174
3451   Topic4    24.857224     vida    29.626524   2.1002  -6.5865
315    Topic4    90.114319      lil   166.164368   1.6638  -5.2985
1684   Topic4    56.380280    color    88.371292   1.8263  -5.7675
1115   Topic4    48.202122      bet    82.299728   1.7408  -5.9242
4041   Topic4    27.470398     como    35.676838   2.0143  -6.4865
2501   Topic4    31.585791    booty    44.678902   1.9289  -6.3469
268    Topic4   114.131935    dance   398.552856   1.0253  -5.0623
456    Topic4   116.734375     stop   420.291962   0.9947  -5.0397
283    Topic4   125.776344     fuck   587.426758   0.7345  -4.9651
365    Topic4    92.754936     shit   389.909729   0.8398  -5.2697
330    Topic4    32.521454      mom    50.556717   1.8345  -6.3177
224    Topic4    67.717522    wrong   257.734283   0.9392  -5.5843
161    Topic4   153.865997     come  1789.325195  -0.1778  -4.7635
335    Topic4    78.397469    nigga   402.861694   0.6389  -5.4378
239    Topic4    81.179947    bitch   441.815247   0.5815  -5.4029
16     Topic4   138.900299       go  1797.216064  -0.2845  -4.8659
45     Topic4   117.598000     baby  1201.734253  -0.0485  -5.0323
14     Topic4   156.394257      get  2729.458740  -0.5837  -4.7472
93     Topic4   132.993500     make  1746.809326  -0.2995  -4.9093
81     Topic4   118.176056     feel  1710.135010  -0.3964  -5.0274
58     Topic4  

In [379]:
vis = pyLDAvis.gensim.prepare(lda_model_4, corpus_4, dict_words_4)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.049548 -0.031771       1        1  45.594128
1      0.035308  0.097619       2        1  23.136765
2      0.049721 -0.058243       3        1  17.371225
3     -0.134576 -0.007605       4        1  13.897882, topic_info=     Category         Freq        Term        Total  loglift  logprob
term                                                                 
834   Default   214.000000         que   214.000000  30.0000  30.0000
201   Default   889.000000        girl   889.000000  29.0000  29.0000
568   Default   272.000000         low   272.000000  28.0000  28.0000
34    Default  1543.000000        baby  1543.000000  27.0000  27.0000
1859  Default   239.000000         doo   239.000000  26.0000  26.0000
377   Default   542.000000       bitch   542.000000  25.0000  25.0000
1981  Default   226.000000         fly   226.000000  24.0000  24.0000
666   Default   498.000000        free   498.000000  23.0000  23.0000
25    Default  1436.000000        take  1436.000000  22.0000  22.0000
406   Default   586.000000        fuck   586.000000  21.0000  21.0000
171   Default   522.000000         put   522.000000  20.0000  20.0000
5     Default   923.000000       could   923.000000  19.0000  19.0000
244   Default   238.000000        wish   238.000000  18.0000  18.0000
1277  Default   471.000000        live   471.000000  17.0000  17.0000
347   Default  1207.000000       wanna  1207.000000  16.0000  16.0000
279   Default   363.000000        high   363.000000  15.0000  15.0000
220   Default   418.000000       nigga   418.000000  14.0000  14.0000
2116  Default   158.000000        lick   158.000000  13.0000  13.0000
257   Default   614.000000         run   614.000000  12.0000  12.0000
913   Default   511.000000       money   511.000000  11.0000  11.0000
657   Default   231.000000        cool   231.000000  10.0000  10.0000
2436  Default   182.000000      fallin   182.000000   9.0000   9.0000
479   Default   137.000000        bird   137.000000   8.0000   8.0000
524   Default   294.000000       stand   294.000000   7.0000   7.0000
134   Default   354.000000  everything   354.000000   6.0000   6.0000
41    Default   603.000000      little   603.000000   5.0000   5.0000
6     Default   950.000000        give   950.000000   4.0000   4.0000
1680  Default   152.000000       river   152.000000   3.0000   3.0000
250   Default   245.000000       happy   245.000000   2.0000   2.0000
982   Default   250.000000        rock   250.000000   1.0000   1.0000
...       ...          ...         ...          ...      ...      ...
1917   Topic4    46.634361       super    63.011681   1.6725  -6.3088
657    Topic4   113.057495        cool   231.538055   1.2566  -5.4233
171    Topic4   173.166260         put   522.197876   0.8696  -4.9969
1680   Topic4    78.422691       river   152.286972   1.3098  -5.7890
1721   Topic4    46.177868        hole    65.606987   1.6223  -6.3187
377    Topic4   165.241180       bitch   542.133789   0.7853  -5.0437
1277   Topic4   151.780334        live   471.759644   0.8394  -5.1287
257    Topic4   173.084534         run   614.302979   0.7067  -4.9974
25     Topic4   282.343018        take  1436.532104   0.3466  -4.5080
1161   Topic4   100.689812        rain   252.478516   1.0542  -5.5391
5      Topic4   199.169235       could   923.290161   0.4396  -4.8570
18     Topic4   218.680420         see  1443.503418   0.0862  -4.7635
294    Topic4    97.632629         sun   305.911560   0.8313  -5.5699
347    Topic4   183.553421       wanna  1207.458862   0.0897  -4.9386
36     Topic4   213.430801        come  1772.607422  -0.1435  -4.7878
107    Topic4   186.786560        feel  1449.547363  -0.0756  -4.9212
94     Topic4   213.291595          go  1995.645630  -0.2626  -4.7885
40     Topic4   236.703110         get  2822.991211  -0.5053  -4.6843
220    Topic4    99.025597       nigga   418.502411   0.5321  -

model_1 = "model_1"

lda_model_1.save(model_1)

# print(corpus[0])
# for item in lda_model[corpus[0]]:
#     print("\n", item)

In [ ]:
# model_2 = "model_2"
# model_3 = "model_3"
# model_4 = "model_4"

# lda_model_2.save(model_2)
# lda_model_3.save(model_3)
# lda_model_4.save(model_4)

from gensim.test.utils import datapath

#save the model to disk
pretrained_model_1 = datapath("pretrained_model_1")
lda_model_1.save(pretrained_model_1)

#input new document to the LDA model, and see the score of classification

In [165]:
#view the topics in LDA model: the keyword in the num_topics topics
# from pprint import pprint
# pprint(lda_model.print_topics())
# doc_lda = lda_model[corpus]

#load the pretrained model from disk
lda_model_1 = LdaModel.load(pretrained_model_1)

In [33]:
# Create the TF-IDF model 
tfidf_1 = models.TfidfModel(corpus_1, smartirs='ntc')
corpus_tfidf_1 = tfidf_1[corpus_1]
lda_tfidf_model_1 = build_LDA_model(dict_words_1, corpus_tfidf_1)

In [34]:
#evaluate the model with tf-idf
print_coherence_perplexity(lda_tfidf_model_1, lemmatized_words_1, dict_words_1, corpus_tfidf_1)


Coherence Score:  0.39085266637305865

Perplexity:  -8.858561083744949


In [255]:
gen_df_tfidf_1 = generate_data(lda_tfidf_model_1, corpus_tfidf_1)
randomForest_LDA(gen_df_tfidf_1, df_1.iloc[:TRAIN_SAMPLE, -1])
print("Classify with Random Forest:")
rf_clf = RanfomForestClassifier()
kfold_classification(gen_df_tfidf_1, Y1, rf_clf)
print("Classify with Complement Naive Bayes classifier:")
cnb_clf = ComplementNB(alpha = 0.1)
kfold_classification(gen_df_tfidf_1, Y1, cnb_clf)

/usr/local/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


[-1  1  1 ...  1 -1 -1]
0.5034904013961605
the 0 th fold split got score: 
0.48778359511343805
the 1 th fold split got score: 


/usr/local/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.5069808027923212
the 2 th fold split got score: 
0.5078534031413613
the 3 th fold split got score: 


/usr/local/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.49214659685863876
the 4 th fold split got score: 
0.4790575916230366


## As we can see that after applying tf-idf model as the corpus input, the cohenrence score increased by 0.01, while the classification score decreased by 0.15.

In [169]:
from sklearn.linear_model import LassoCV
#it's said the best model will be chosen by cross-validation


## Improve upon this model by using Mallet's version of LDA algorithm, which always gives a better quality of topics than Gensim's inbuilt version of the LDA algorithm.

In [38]:
#use mallet's implementation of LDA
mallet_path = "../mallet-2.0.8/bin/mallet"
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus_1, num_topics=NUM_TOPICS, id2word=dict_words_1)
# evaluate the mallet model: Compute Coherence Score
coherence_model = CoherenceModel(model = ldamallet, texts = lemmatized_words_1, dictionary = dict_words_1, coherence = 'c_v')
coherence_lda = coherence_model.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.3783316243882765


#classify the dataset using output of mallet's LDA model
gen_df_mallet_1 = generate_data(ldamallet, corpus_1)
randomForest_LDA(gen_df_mallet_1, df_1.iloc[:TRAIN_SAMPLE, -1])
print("Classfy using data generated by Mallet's LDA with Random Forest Classifier:")
## AttributeError: 'LdaMallet' object has no attribute 'get_document_topics'

In [40]:
#try tf-idf on mallet
ldamallet_tfidf = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus_tfidf_1, num_topics=NUM_TOPICS, id2word=dict_words_1)
# evaluate the mallet model: Compute Coherence Score
coherence_model = CoherenceModel(model = ldamallet_tfidf, texts = lemmatized_words_1, dictionary = dict_words_1, coherence = 'c_v')
coherence_lda = coherence_model.get_coherence()
print('\nCoherence Score: ', coherence_lda)

/usr/local/lib/python3.7/site-packages/gensim/models/wrappers/ldamallet.py:380: RuntimeWarning: invalid value encountered in true_divide
  return topics / topics.sum(axis=1)[:, None]



Coherence Score:  0.45514237865985774


How to infer pyLDAvis's output?

Each bubble represents a topic. The larger the bubble, the more prevalent is that topic.

A good topic model will have fairly big, non-overlapping bubbles scattered throughout the chart instead of being clustered in one quadrant.

A model with too many topics, will typically have many overlaps, small sized bubbles, clustered in one region of the chart

So this model ... maybe use fewer number of topics

You could determine the number of topics using prior knowledge of the number of natural topics in the document

In [ ]:
#We will focus on how to arrive at the optimal number of topics given any large corpus of text
#TODO: I really want to know what can gridsearch do in finding the optimal number of topics

In [ ]:
# Show the TF-IDF weights
for doc in tfidf[corpus]:
    print([[dict_words[id], np.around(freq, decimals=2)] for id, freq in doc])